In [17]:
#!pip install opencv-python
#!pip install keras_vis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 186.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import mlflow

from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2


In [8]:
mlops_server_uri = 'https://david-rem-jedha-final-project-mlops.hf.space'
model_path = 'models:/lung_7_classes/2'
#model_path = "runs:/73f7af9ab4be4dd1be44b4653bb2bf06/model"

mlflow.set_tracking_uri(mlops_server_uri)
keras_model = mlflow.keras.load_model(model_path)

# Vérifier si le modèle est bien chargé
print(type(keras_model))  # Doit être <class 'keras.engine.functional.Functional'> ou <class 'tensorflow.keras.Model'>

base_model = keras_model.layers[1]

<class 'keras.src.models.sequential.Sequential'>


In [54]:
for layer in keras_model.layers:
    print(layer.name)
    
print('\n')

for layer in keras_model.layers[1].layers:
    print(layer.name)

print('\n')

last_conv_layer_name = keras_model.layers[1].layers[-1].name
print(last_conv_layer_name)

conv2d_190
InceptionV3
conv2d_191
flatten_1
dense_1


input_layer_2
conv2d_96
batch_normalization_94
activation_94
conv2d_97
batch_normalization_95
activation_95
conv2d_98
batch_normalization_96
activation_96
max_pooling2d_4
conv2d_99
batch_normalization_97
activation_97
conv2d_100
batch_normalization_98
activation_98
max_pooling2d_5
conv2d_104
batch_normalization_102
activation_102
conv2d_102
conv2d_105
batch_normalization_100
batch_normalization_103
activation_100
activation_103
average_pooling2d_9
conv2d_101
conv2d_103
conv2d_106
conv2d_107
batch_normalization_99
batch_normalization_101
batch_normalization_104
batch_normalization_105
activation_99
activation_101
activation_104
activation_105
mixed0
conv2d_111
batch_normalization_109
activation_109
conv2d_109
conv2d_112
batch_normalization_107
batch_normalization_110
activation_107
activation_110
average_pooling2d_10
conv2d_108
conv2d_110
conv2d_113
conv2d_114
batch_normalization_106
batch_normalization_108
batch_normalization_111
ba

In [9]:
keras_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_94 (Conv2D)              │ (None, 128, 128, 3)    │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ InceptionV3 (Functional)        │ (None, 2, 2, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │        57,351 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,852,860 (90.99 MB)

 Trainable params: 1,992,717 (7.60 MB)

 Non-trainable params: 19,867,424 (75.79 MB)

 Optimizer params: 1,992,719 (7.60 MB)

In [28]:
#keras_model.layers[-3]

<Conv2D name=conv2d_191, built=True>

In [16]:
# The Grad-CAM algorithm 
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (224, 224, 1)
    array = keras.preprocessing.image.img_to_array(img)
    print(array.shape)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 224, 224, 1)
    array = np.expand_dims(array, axis=0)
    print(array.shape)
    return array


def make_gradcam_heatmap(
    img_array, base_model, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = base_model.get_layer(last_conv_layer_name)
    first_conv_layer = model.get_layer('conv2d_94')
    #last_conv_layer = model.get_layer(last_conv_layer_name)
    #print(last_conv_layer)
    first_conv_layer_model = tf.keras.Model(base_model.inputs, first_conv_layer.output)
    last_conv_layer_model = tf.keras.Model(first_conv_layer_model.inputs, last_conv_layer.output)
    #last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)
    #print(last_conv_layer_model)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    print(classifier_layer_names[0])
    first_input = tf.keras.Input(shape=img_array.shape)
    #classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:]) #last_conv_layer.output.shape[1:] #img_array.shape
    x = first_input
    for layer_name in classifier_layer_names:
        print(layer_name)
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(first_input, x) #classifier_input

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        #last_conv_layer_output = last_conv_layer_model(img_array)
        last_conv_layer_input = first_conv_layer_model(img_array)
        last_conv_layer_output = last_conv_layer_model(last_conv_layer_input)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [29]:
#array = get_img_array("./image_0007.jpg", (224, 224, 1))
img = cv2.imread("./image_0007.jpg", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (128, 128)) # Ajustez selon votre modèle
array = np.expand_dims(img, axis=-1) # Ajustez selon votre modèle
#array = np.expand_dims(np.expand_dims(img, axis=-1), axis=0)
#img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
#array = np.expand_dims(img_color, axis=0) 
print(array.shape)

classifier_layer_names = [layer.name for layer in keras_model.layers]
print(classifier_layer_names)
last_conv_layer_name = keras_model.layers[1].layers[-1].name
print(last_conv_layer_name)

heatmap = make_gradcam_heatmap(
      array, base_model, keras_model, last_conv_layer_name, classifier_layer_names
      #array, base_model, keras_model, "conv2d_191", classifier_layer_names
    )



(128, 128, 1)
['conv2d_94', 'InceptionV3', 'flatten', 'dense']
mixed10
conv2d_94
conv2d_94
InceptionV3
flatten
dense


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(128, 128, 1))
  warnings.warn(msg)


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input [[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]. Expected shape (None, 128, 128, 3), but input has incompatible shape (128, 128, 1)[0m

Arguments received by Functional.call():
  • inputs=array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
  

In [30]:
#from PIL import Image
#img = Image.open("./image_0007.jpg")
#img_resized = img.resize((128, 128))


apply_gradcam("./image_0007.jpg", keras_model, "conv2d_191")

(128, 128, 1)
conv2d_190
InceptionV3
conv2d_191
flatten_1
dense_1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_682']
Received: inputs=Tensor(shape=(128, 128, 1))
  warnings.warn(msg)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "InceptionV3" is incompatible with the layer: expected shape=(None, 128, 128, 3), found shape=(128, 128, 1, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(128, 128, 1, 1), dtype=float32)
  • training=None
  • mask=None